In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load file

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)
df.show()

In [ ]:
df = df.dropna()
df = df.dropDuplicates()
df.count()

In [ ]:
df.printSchema()

In [ ]:
# from pyspark.sql.functions import to_date
reviewDF = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviewDF.show(5)

In [ ]:
productsDF = df.select(['product_id', 'product_title']).drop_duplicates()
productsDF.show(5)

In [ ]:
customersDF = df.groupby('customer_id').agg({'customer_id': 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customersDF.show(5)

In [ ]:
vineDF = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vineDF.show(5)

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-2.cofijofykqn6.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "password",
          "driver": "org.postgresql.Driver"}

In [ ]:
reviewDF.write.jdbc(url=jdbc_url, table='Review_id_Table', mode=mode, properties=config)

In [ ]:
productsDF.write.jdbc(url=jdbc_url, table='Products', mode=mode, properties=config)

In [ ]:
customersDF.write.jdbc(url=jdbc_url, table='Customers', mode=mode, properties=config)

In [ ]:
vineDF.write.jdbc(url=jdbc_url, table='Vine_Table', mode=mode, properties=config)